# Data Requests 

## Setup

In [1]:
# Import modules
import time

import pandas as pd

import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys


In [2]:
# API connexion
client_id = 'cec8979c027344f98b471a991aa415ad'
client_secret = 'ad69bcba55b349f98c1344b006c708bd'

client_credentials_manager = SpotifyClientCredentials(client_id, client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)


## Get playlists infos 

In [ ]:
# API scrapping
# Set category IDs
category_ids = [
    '0JQ5DAqbMKFNQ0fGp4byGU', '0JQ5DAqbMKFx0uLQR2okcc', '0JQ5DAqbMKFFtlLYUHv8bT', '0JQ5DAqbMKFzHmL4tf05da', '0JQ5DAqbMKFLjmiZRss79w',
    '0JQ5DAqbMKFAUsdyVjCQuL', '0JQ5DAqbMKFziKOShCi009', '0JQ5DAqbMKFQ1UFISXj59F', '0JQ5DAqbMKFSCjnQr8QZ3O', '0JQ5DAqbMKFLb2EqgLtpjC', 
    '0JQ5DAqbMKFQiK2EHwyjcU', '0JQ5DAqbMKFObNLOHydSW8', '0JQ5DAqbMKFPrEiAOxgac3', '0JQ5DAqbMKFKLfwjuJMoNC', '0JQ5DAqbMKFRY5ok2pxXJ0', 
    '0JQ5DAqbMKFHOzuVTgTizF', '0JQ5DAqbMKFP9UNkavIGrM', '0JQ5DAqbMKFIVNxQgRNSg0', '0JQ5DAqbMKFFzDl7qN9Apr', '0JQ5DAqbMKFCuoRTxhYWow', 
    '0JQ5DAqbMKFFoimhOqWzLB', '0JQ5DAqbMKFIRybaNTYXXy', '0JQ5DAqbMKFPw634sFwguI', '0JQ5DAqbMKFJw7QLnM27p6', '0JQ5DAqbMKFCbimwdOYlsl', 
    '0JQ5DAqbMKFy78wprEpAjl', '0JQ5DAqbMKFF9bY76LXmfI', '0JQ5DAqbMKFImHYGo3eTSg', '0JQ5DAqbMKFFsW9N8maB6z', '0JQ5DAqbMKFCfObibaOZbv', 
    '0JQ5DAqbMKFGnsSfvg90Wo', '0JQ5DAqbMKFQ00XGBls6ym', '0JQ5DAqbMKFCWjUTdzaG0e', '0JQ5DAqbMKFRKBHIxJ5hMm', '0JQ5DAqbMKFAJ5xb0fwo9m', 
    '0JQ5DAqbMKFGvOw3O4nLAf', '0JQ5DAqbMKFxXaXKP7zcDp', '0JQ5DAqbMKFDkd668ypn6O', '0JQ5DAqbMKFRieVZLLoo9m', '0JQ5DAqbMKFI3pNLtYMD9S', 
    '0JQ5DAqbMKFEOEBCABAxo9', '0JQ5DAqbMKFEC4WFtoNRpw', '0JQ5DAqbMKFAjfauKLOZiv', '0JQ5DAqbMKFOOxftoKZxod', '0JQ5DAqbMKFJKoGyUMo2hE', 
    '0JQ5DAqbMKFEZPnFQSFB1T', '0JQ5DAqbMKFDXXwE9BDJAr', '0JQ5DAqbMKFA6SOHvT3gck', '0JQ5DAqbMKFIpEuaCnimBj', '0JQ5DAqbMKFAXlCG6QvYQ4', 
    '0JQ5DAqbMKFDBgllo2cUIN', '0JQ5DAqbMKFQIL0AXnG5AK', '0JQ5DAqbMKFOzQeOmemkuw', '0JQ5DAqbMKFBweu2euf3Sj', '0JQ5DAqbMKFAQy4HL4XU2D'
]

# Store missing categories
missing_categories = []

# Loop over all categories and get playlists
playlists = []
for category_id in category_ids:
    try:
        # Get first 50 playlists in category
        response = sp.category_playlists(category_id, limit=50)
        category_playlists = response["playlists"]["items"]
        
        # Check if there are more playlists in category
        while response["playlists"]["next"]:
            response = sp.next(response["playlists"])
            category_playlists += response["playlists"]["items"]
        
        # Append category playlists to list
        for playlist in category_playlists:
            playlist_id = playlist["id"]
            playlist_name = playlist["name"]
            playlist_owner = playlist["owner"]["display_name"]
            playlists.append({
                "category_id": category_id, 
                "playlist_id": playlist_id, 
                "playlist_name": playlist_name,
                "playlist_owner": playlist_owner
            })
    except: 
        missing_categories.append(category_id)
        print(f"Category with ID {category_id} doesn't exist.")

# Create dataframe from playlists
df_playlists = pd.DataFrame(playlists, columns=["category_id", "playlist_id", "playlist_name", "playlist_owner"])

print(df_playlists)


In [ ]:
# HTML scraping for Charts playlists
# URLs to scrape
urls = [
    'https://open.spotify.com/genre/section0JQ5DAzQHECxDlYNI6xD1h',
    'https://open.spotify.com/genre/section0JQ5DAzQHECxDlYNI6xD1i',
    'https://open.spotify.com/genre/section0JQ5DAzQHECxDlYNI6xD1j'
]

# Set up the web driver
driver = webdriver.Safari(executable_path='/usr/bin/safaridriver')

top_playlists = []

for url in urls:
    driver.get(url)
    time.sleep(5)  # Wait for playlists to load

    # Find playlist elements by class name
    playlists = driver.find_elements(By.CLASS_NAME, 'Box__BoxComponent-sc-y4nds-0')
    last_playlist = playlists[-1]

    # Scroll to load all playlists
    while True:
        driver.execute_script("arguments[0].scrollIntoView();", last_playlist)
        time.sleep(2)
        new_playlists = driver.find_elements(By.CLASS_NAME, 'Box__BoxComponent-sc-y4nds-0')
        if new_playlists[-1] != last_playlist:
            last_playlist = new_playlists[-1]
            playlists = new_playlists
        else:
            break

    # Extract playlist information
    for playlist in playlists:
        # Get aria-labelledby attribute
        aria_label = playlist.get_attribute('aria-labelledby')
        
        # Extract playlist ID from aria-labelledby
        if aria_label and "spotify:playlist:" in aria_label:
            playlist_id = aria_label.split("spotify:playlist:")[1].split()[0]
            top_playlists.append({
                "category_id": '0JQ5DAudkNjCgYMM0TZXDw', 
                "playlist_id": playlist_id, 
                "playlist_owner": 'Spotify'
            })

# Quit the driver
driver.quit()

# Convert data to DataFrame and print
df_top_playlists = pd.DataFrame(top_playlists, columns=["category_id", "playlist_id", "playlist_owner"])
print(df_top_playlists)


In [11]:
# Parse playlists info into excel
# Combine the manual and API playlists into one DataFrame
df_combined = pd.concat([df_playlists, df_top_playlists], ignore_index=True)
df_combined.to_excel("playlists_info.xlsx", index=False)

# Drop duplicates based on the "playlist_id" column, keeping the first occurrence
df_cleaned = df_combined.drop_duplicates(subset='playlist_id', keep='first')
df_cleaned.to_excel("playlists_info_clean.xlsx", index=False)


## Get tracks info

### Setup

In [3]:
# Load data
df = pd.read_excel('playlists_info_clean.xlsx')

# Remove playlists with "This is" and "Written By" in the name
df = df[~df['playlist_name'].str.contains('This Is', na=False)]
df = df[~df['playlist_name'].str.contains('Written By', na=False)]


In [4]:
# Define scraping functions
# Define a function to get all the tracks in a playlist, handling the API limit
def get_all_playlist_tracks(playlist_id):
    results = sp.playlist_tracks(playlist_id)
    tracks = results['items']
    while results['next']:
        results = sp.next(results)
        tracks.extend(results['items'])
    return tracks

# Define a function to get all the audio features for a set of tracks, handling the API limit
def get_all_audio_features(track_ids):
    audio_features = []
    for i in range(0, len(track_ids), 100):
        batch = track_ids[i:i+100]
        try:
            features = sp.audio_features(batch)
        except:
            continue
        audio_features.extend(features)
    return audio_features

def get_playlist_track_info(playlist_id):
    # Get the playlist info
    playlist_info = sp.playlist(playlist_id)
    if not playlist_info:
        return None
    
    playlist_name = playlist_info['name']
    playlist_followers = playlist_info['followers']['total']
    playlist_owner = playlist_info['owner']['display_name']
    
    # Get all the tracks in the playlist
    playlist_tracks = get_all_playlist_tracks(playlist_id)
    track_ids = [track['track']['id'] for track in playlist_tracks if track['track'] is not None]
    
    if not track_ids:
        return None
    
    # Get the track info in batches of 50
    track_info = []
    for i in range(0, len(track_ids), 50):
        batch_tracks = track_ids[i:i+50]
        tracks = sp.tracks(batch_tracks)['tracks']
        track_info.extend(tracks)

    if track_info is not None:
        track_names = [track['name'] for track in track_info]
        track_popularities = [track['popularity'] for track in track_info]
        track_release_dates = [track['album']['release_date'] for track in track_info]
        artist_ids = [track['artists'][0]['id'] for track in track_info]
        
        artist_info = []
        for i in range(0, len(artist_ids), 50):
            batch_artists = artist_ids[i:i+50]
            artists = sp.artists(batch_artists)['artists']
            artist_info.extend(artists)
        
        artist_names = [artist['name'] for artist in artist_info]
        artist_popularities = [artist['popularity'] for artist in artist_info]

        # Combine everything into a DataFrame
        playlist_df = pd.DataFrame({
            'playlist_name': playlist_name,
            'playlist_followers': playlist_followers,
            'playlist_owner': playlist_owner,
            'track_name': track_names,
            'track__popularity': track_popularities,
            'track_release_date': track_release_dates,
            'artist_name': artist_names,
            'artist_popularity': artist_popularities
        })
    else:
        return None

    # Get the audio features
    audio_features = get_all_audio_features(track_ids)
    if audio_features is None:
        return None
    
    # Add the audio features to the DataFrame
    try:
        audio_features_df = pd.DataFrame(audio_features)
        playlist_df = pd.concat([playlist_df, audio_features_df], axis=1)
    except:
        return None
    
    return playlist_df

# Define a function to loop over all the playlists and combine the results
def get_all_playlist_track_info(playlist_ids):
    playlist_dfs = []
    for playlist_id in playlist_ids:
        print(f"Processing playlist: {playlist_id}")
        playlist_df = get_playlist_track_info(playlist_id)
        if playlist_df is not None:
            playlist_dfs.append(playlist_df)
    all_playlist_df = pd.concat(playlist_dfs, ignore_index=True)
    return all_playlist_df


### Scrape the data for each category

In [ ]:
# Tops
category_id = "0JQ5DAudkNjCgYMM0TZXDw"

filtered_df = df[df['category_id'] == category_id]
playlist_ids = filtered_df['playlist_id'].tolist()

df_tops = get_all_playlist_track_info(playlist_ids)
df_tops.to_excel("df_tops.xlsx", index = False)


In [ ]:
# Afro
category_id = "0JQ5DAqbMKFNQ0fGp4byGU"

filtered_df = df[df['category_id'] == category_id]
playlist_ids = filtered_df['playlist_id'].tolist()

df_pop = get_all_playlist_track_info(playlist_ids)
df_pop.to_excel("df_afro.xlsx", index = False)


In [ ]:
# A la Maison
category_id = "0JQ5DAqbMKFx0uLQR2okcc"

filtered_df = df[df['category_id'] == category_id]
playlist_ids = filtered_df['playlist_id'].tolist()

df_maison = get_all_playlist_track_info(playlist_ids)
df_maison.to_excel("df_maison.xlsx", index = False)


In [ ]:
# Alternative
category_id = "0JQ5DAqbMKFFtlLYUHv8bT"

filtered_df = df[df['category_id'] == category_id]
playlist_ids = filtered_df['playlist_id'].tolist()

df_alternative = get_all_playlist_track_info(playlist_ids)
df_alternative.to_excel("df_alternative.xlsx", index = False)


In [ ]:
# Ambiance
category_id = "0JQ5DAqbMKFzHmL4tf05da"

filtered_df = df[df['category_id'] == category_id]
playlist_ids = filtered_df['playlist_id'].tolist()

df_ambiance = get_all_playlist_track_info(playlist_ids)
df_ambiance.to_excel("df_ambiance.xlsx", index = False)


In [ ]:
# Ambient
category_id = "0JQ5DAqbMKFLjmiZRss79w"

filtered_df = df[df['category_id'] == category_id]
playlist_ids = filtered_df['playlist_id'].tolist()

df_ambient = get_all_playlist_track_info(playlist_ids)
df_ambient.to_excel("df_ambient.xlsx", index = False)


In [ ]:
# Amour
category_id = "0JQ5DAqbMKFAUsdyVjCQuL"

filtered_df = df[df['category_id'] == category_id]
playlist_ids = filtered_df['playlist_id'].tolist()

df_amour = get_all_playlist_track_info(playlist_ids)
df_amour.to_excel("df_amour.xlsx", index = False)


In [ ]:
# Anime
category_id = "0JQ5DAqbMKFziKOShCi009"

filtered_df = df[df['category_id'] == category_id]
playlist_ids = filtered_df['playlist_id'].tolist()

df_anime = get_all_playlist_track_info(playlist_ids)
df_anime.to_excel("df_anime.xlsx", index = False)


In [ ]:
# Arabe
category_id = "0JQ5DAqbMKFQ1UFISXj59F"

filtered_df = df[df['category_id'] == category_id]
playlist_ids = filtered_df['playlist_id'].tolist()

df_arabe = get_all_playlist_track_info(playlist_ids)
df_arabe.to_excel("df_arabe.xlsx", index = False)


In [5]:
# Bien etre
category_id = "0JQ5DAqbMKFLb2EqgLtpjC"

filtered_df = df[df['category_id'] == category_id]
playlist_ids = filtered_df['playlist_id'].tolist()

df_bienetre = get_all_playlist_track_info(playlist_ids)
df_bienetre.to_excel("df_bienetre.xlsx", index = False)


Processing playlist: 37i9dQZF1DX8LIZNmf5qj3
Processing playlist: 37i9dQZF1DWTwnEm1IYyoj
Processing playlist: 37i9dQZF1DXdbQ2qAyjZ1R
Processing playlist: 37i9dQZF1DXcdw9vRvRM79
Processing playlist: 37i9dQZF1DWSGaMpjluQpy
Processing playlist: 37i9dQZF1DXcrbQpZuKaRH
Processing playlist: 37i9dQZF1DX1KVBf2zZZ2X
Processing playlist: 37i9dQZF1DX7NeseHaSkSz
Processing playlist: 37i9dQZF1DXcECZl1cQzi5
Processing playlist: 37i9dQZF1DX9j444F9NCBa
Processing playlist: 37i9dQZF1DWYY963019MQr
Processing playlist: 37i9dQZF1DWUvHZA1zLcjW
Processing playlist: 37i9dQZF1DWV7EzJMK2FUI
Processing playlist: 37i9dQZF1DWSADWNdZfn11
Processing playlist: 37i9dQZF1DWTcEjayzrZ4x
Processing playlist: 37i9dQZF1DX4uB43NNq1P7
Processing playlist: 37i9dQZF1DXdPDLmy88MDk
Processing playlist: 37i9dQZF1DX7K31D69s4M1
Processing playlist: 37i9dQZF1DWXlntHTG08uC
Processing playlist: 37i9dQZF1DX9JGJTJ2WFXi
Processing playlist: 37i9dQZF1DWTvEFX6xtoQd
Processing playlist: 37i9dQZF1DWSRkksm0qURy
Processing playlist: 37i9dQZF1DX

In [6]:
# Blues
category_id = "0JQ5DAqbMKFQiK2EHwyjcU"

filtered_df = df[df['category_id'] == category_id]
playlist_ids = filtered_df['playlist_id'].tolist()

df_blues = get_all_playlist_track_info(playlist_ids)
df_blues.to_excel("df_blues.xlsx", index = False)


Processing playlist: 37i9dQZF1DXd9rSDyQguIk
Processing playlist: 37i9dQZF1DXcu3QLJudo4X
Processing playlist: 37i9dQZF1DX9GnjlK17CLK
Processing playlist: 37i9dQZF1DX9GSZDbrndTa
Processing playlist: 37i9dQZF1DWWhiW4fdIska
Processing playlist: 37i9dQZF1DXbUXCbzCkRVP
Processing playlist: 37i9dQZF1DX7KwBiutIRku
Processing playlist: 37i9dQZF1DXbxvVclJxhEJ
Processing playlist: 37i9dQZF1DX4snGSoRjGMW
Processing playlist: 37i9dQZF1DX5XEAHcCzWcH
Processing playlist: 37i9dQZF1DXb3MZdETGqKB
Processing playlist: 37i9dQZF1DXcnkReojaCnV
Processing playlist: 37i9dQZF1DX7Y7BqFok9IQ
Processing playlist: 37i9dQZF1DX0QNpebF7rcL
Processing playlist: 37i9dQZF1DXcFk5r8uS3l2
Processing playlist: 37i9dQZF1DWSKpvyAAcaNZ
Processing playlist: 37i9dQZF1DX9stbPFTxeaB
Processing playlist: 37i9dQZF1DXdeoRwG5yUKg
Processing playlist: 37i9dQZF1DWT1RLff7HxDm
Processing playlist: 37i9dQZF1DX7b1e7B17gg7
Processing playlist: 37i9dQZF1DWWyCCtyRAvGr
Processing playlist: 37i9dQZF1DX2Pqxel6mitp
Processing playlist: 37i9dQZF1DX

In [7]:
# Caraibes
category_id = "0JQ5DAqbMKFObNLOHydSW8"

filtered_df = df[df['category_id'] == category_id]
playlist_ids = filtered_df['playlist_id'].tolist()

df_caraibes = get_all_playlist_track_info(playlist_ids)
df_caraibes.to_excel("df_caraibes.xlsx", index = False)


Processing playlist: 37i9dQZF1DX9QYRS3EMTFh
Processing playlist: 37i9dQZF1DWZjVBCh3y4ph
Processing playlist: 37i9dQZF1DX1yXxWD7WGcm
Processing playlist: 37i9dQZF1DWYtKpmml7moA
Processing playlist: 37i9dQZF1DWSiyIBdVQrkk
Processing playlist: 37i9dQZF1DXan38dNVDdl4
Processing playlist: 37i9dQZF1DX7r3u5HAAB4A
Processing playlist: 37i9dQZF1DX7GUbRHVEX42
Processing playlist: 37i9dQZF1DWWDLvD3P5wOG
Processing playlist: 37i9dQZF1DX0cNYCfLOrsu


In [8]:
# Classique
category_id = "0JQ5DAqbMKFPrEiAOxgac3"

filtered_df = df[df['category_id'] == category_id]
playlist_ids = filtered_df['playlist_id'].tolist()

df_classique = get_all_playlist_track_info(playlist_ids)
df_classique.to_excel("df_classique.xlsx", index = False)


Processing playlist: 37i9dQZF1DWWEJlAGA9gs0
Processing playlist: 37i9dQZF1DWV0gynK7G6pD
Processing playlist: 37i9dQZF1DX17GkScaAekA
Processing playlist: 37i9dQZF1DWTi57LiHYTmH
Processing playlist: 37i9dQZF1DWUPafHP1BJw1
Processing playlist: 37i9dQZF1DX5Lm1ZiObdc3
Processing playlist: 37i9dQZF1DX9OZisIoJQhG
Processing playlist: 37i9dQZF1DWUqIzZNMSCv3
Processing playlist: 37i9dQZF1DWVFeEut75IAL
Processing playlist: 37i9dQZF1DX7Ope5RE4ePQ
Processing playlist: 37i9dQZF1DX8Sz1gsYZdwj
Processing playlist: 37i9dQZF1DWYkztttC1w38
Processing playlist: 37i9dQZF1DX8qVlPzYQL5d
Processing playlist: 37i9dQZF1DX9dX3aBjsxqd
Processing playlist: 37i9dQZF1DXdTVW77weWfh
Processing playlist: 37i9dQZF1DX0Aaer4Jzfgm
Processing playlist: 37i9dQZF1DX1tz6EDao8it
Processing playlist: 37i9dQZF1DXdfOcg1fm0VG
Processing playlist: 37i9dQZF1DWW7gj0FcGEx6
Processing playlist: 37i9dQZF1DWVJd52AtCSiR
Processing playlist: 37i9dQZF1DX1xWMTR2Zu8L
Processing playlist: 37i9dQZF1DWXahxq4Q8el6
Processing playlist: 37i9dQZF1DW

In [9]:
# Country
category_id = "0JQ5DAqbMKFKLfwjuJMoNC"

filtered_df = df[df['category_id'] == category_id]
playlist_ids = filtered_df['playlist_id'].tolist()

df_country = get_all_playlist_track_info(playlist_ids)
df_country.to_excel("df_country.xlsx", index = False)


Processing playlist: 37i9dQZF1DX1lVhptIYRda
Processing playlist: 37i9dQZF1DWZBCPUIUs2iR
Processing playlist: 37i9dQZF1DWVpjAJGB70vU
Processing playlist: 37i9dQZF1DXaiEFNvQPZrM
Processing playlist: 37i9dQZF1DWW7RgkOJG32Y
Processing playlist: 37i9dQZF1DWYnwbYQ5HnZU
Processing playlist: 37i9dQZF1DX8WMG8VPSOJC
Processing playlist: 37i9dQZF1DWVn8zvR5ROMB
Processing playlist: 37i9dQZF1DX7hnECllVaUq
Processing playlist: 37i9dQZF1DXdxUH6sNtcDe
Processing playlist: 37i9dQZF1DWXdiK4WAVRUW
Processing playlist: 37i9dQZF1DX6RCydf9ytsj
Processing playlist: 37i9dQZF1DWYP5PUsVbso9
Processing playlist: 37i9dQZF1DX7CGYgLhqwu5
Processing playlist: 37i9dQZF1DWWnpcjfCqaW0
Processing playlist: 37i9dQZF1DX0vVAWnG6fKT


In [10]:
# Cuisine
category_id = "0JQ5DAqbMKFRY5ok2pxXJ0"

filtered_df = df[df['category_id'] == category_id]
playlist_ids = filtered_df['playlist_id'].tolist()

df_cuisine = get_all_playlist_track_info(playlist_ids)
df_cuisine.to_excel("df_cuisine.xlsx", index = False)


Processing playlist: 37i9dQZF1DWVUxkQFrGCkK
Processing playlist: 37i9dQZF1DX915ogFalrko
Processing playlist: 37i9dQZF1DWZLN2cXno63R
Processing playlist: 37i9dQZF1DX4xuWVBs4FgJ
Processing playlist: 37i9dQZF1DWSy1uHq5qF1L
Processing playlist: 37i9dQZF1DX7ovYHwmjqZK
Processing playlist: 37i9dQZF1DWTALrdBtcXjw


In [11]:
# Dance
category_id = "0JQ5DAqbMKFHOzuVTgTizF"

filtered_df = df[df['category_id'] == category_id]
playlist_ids = filtered_df['playlist_id'].tolist()

df_dance = get_all_playlist_track_info(playlist_ids)
df_dance.to_excel("df_dance.xlsx", index = False)


Processing playlist: 37i9dQZF1DX5wB72P2sVsT
Processing playlist: 37i9dQZF1DX9ND1QF5hZNF
Processing playlist: 37i9dQZF1DWWY64wDtewQt
Processing playlist: 37i9dQZF1DWWn9pcJIAKFl
Processing playlist: 37i9dQZF1DWWz3YvZd9NIz
Processing playlist: 37i9dQZF1DX9cbNxuNYT3d
Processing playlist: 37i9dQZF1DXa6qtcwTgOjM
Processing playlist: 37i9dQZF1DXa3EUr6VSsMJ
Processing playlist: 37i9dQZF1DX6VdMW310YC7
Processing playlist: 37i9dQZF1DX4dyzvuaRJ0n
Processing playlist: 37i9dQZF1DX7ZUug1ANKRP
Processing playlist: 37i9dQZF1DX0BcQWzuB7ZO
Processing playlist: 37i9dQZF1DX3d2wagqzwmM
Processing playlist: 37i9dQZF1DX12meDmp2yzx
Processing playlist: 37i9dQZF1DWW668oFRU96k
Processing playlist: 37i9dQZF1DX8tZsk68tuDw
Processing playlist: 37i9dQZF1DX6J5NfMJS675
Processing playlist: 37i9dQZF1DX1zcQs2sF77z
Processing playlist: 37i9dQZF1DXaWf8ZIHreXF
Processing playlist: 37i9dQZF1DWSj4n2q6ZYVe
Processing playlist: 37i9dQZF1DX802IXCAaWtY
Processing playlist: 37i9dQZF1DXcytuPF2IzEB
Processing playlist: 37i9dQZF1DX

In [12]:
# Dancehall
category_id = "0JQ5DAqbMKFP9UNkavIGrM"

filtered_df = df[df['category_id'] == category_id]
playlist_ids = filtered_df['playlist_id'].tolist()

df_dancehall = get_all_playlist_track_info(playlist_ids)
df_dancehall.to_excel("df_dancehall.xlsx", index = False)


Processing playlist: 37i9dQZF1DX0rM1NjYKMJa


In [13]:
# Decennies
category_id = "0JQ5DAqbMKFIVNxQgRNSg0"

filtered_df = df[df['category_id'] == category_id]
playlist_ids = filtered_df['playlist_id'].tolist()

df_decennies = get_all_playlist_track_info(playlist_ids)
df_decennies.to_excel("df_decennies.xlsx", index = False)


Processing playlist: 37i9dQZF1DWWl7MndYYxge
Processing playlist: 37i9dQZF1DXacPj7eARo6k
Processing playlist: 37i9dQZF1DWWGI3DKkKGzJ
Processing playlist: 37i9dQZF1DX8E06AbSENEw
Processing playlist: 37i9dQZF1DX7LGssahBoms
Processing playlist: 37i9dQZF1DX4UtSsGT1Sbe
Processing playlist: 37i9dQZF1DX7e8TjkFNKWH
Processing playlist: 37i9dQZF1DX4o1oenSJRJd
Processing playlist: 37i9dQZF1DXbTxeAdrVG2l
Processing playlist: 37i9dQZF1DX6xnkAwJX7tn
Processing playlist: 37i9dQZF1DX69KJk2S04Hp
Processing playlist: 37i9dQZF1DWSvv6VnIb3i0
Processing playlist: 37i9dQZF1DWXLbJb1PtkXq
Processing playlist: 37i9dQZF1DX0zyaFj8e28t
Processing playlist: 37i9dQZF1DX7W8X7B8YNLZ
Processing playlist: 37i9dQZF1DWZY8dtEUdfJ4
Processing playlist: 37i9dQZF1DWZZvzq1xsEuc


In [14]:
# Detente
category_id = "0JQ5DAqbMKFFzDl7qN9Apr"

filtered_df = df[df['category_id'] == category_id]
playlist_ids = filtered_df['playlist_id'].tolist()

df_detente = get_all_playlist_track_info(playlist_ids)
df_detente.to_excel("df_detente.xlsx", index = False)


Processing playlist: 37i9dQZF1DWZfpLyFUtQnu
Processing playlist: 37i9dQZF1DWY6UWUOwj4BO
Processing playlist: 37i9dQZF1DWYoYGBbGKurt
Processing playlist: 37i9dQZF1DXdLK5wjKyhVm
Processing playlist: 37i9dQZF1DX4t95PAs1EpY
Processing playlist: 37i9dQZF1DX0SM0LYsmbMT
Processing playlist: 37i9dQZF1DX4nNmLlb3JR2
Processing playlist: 37i9dQZF1DX3SNr5BeQZSd
Processing playlist: 37i9dQZF1DWVqfgj8NZEp1
Processing playlist: 37i9dQZF1DXdk2DZvqpqJY
Processing playlist: 37i9dQZF1DXbOVU4mpMJjh
Processing playlist: 37i9dQZF1DWWfxnl2EyBbd
Processing playlist: 37i9dQZF1DX08n7V52Ynyb
Processing playlist: 37i9dQZF1DX949uWWpmTjT
Processing playlist: 37i9dQZF1DX2vYju3i0lNX
Processing playlist: 37i9dQZF1DX2m4xSMtooam
Processing playlist: 37i9dQZF1DX6lttj7ulLd6
Processing playlist: 37i9dQZF1DWUN87b7HRrCS
Processing playlist: 37i9dQZF1DX2TRYkJECvfC
Processing playlist: 37i9dQZF1DX3qCx5yEZkcJ
Processing playlist: 37i9dQZF1DXc8kgYqQLMfH
Processing playlist: 37i9dQZF1DX8V4BE7YIpvE
Processing playlist: 37i9dQZF1DW

In [15]:
# Dormir
category_id = "0JQ5DAqbMKFCuoRTxhYWow"

filtered_df = df[df['category_id'] == category_id]
playlist_ids = filtered_df['playlist_id'].tolist()

df_dormir = get_all_playlist_track_info(playlist_ids)
df_dormir.to_excel("df_dormir.xlsx", index = False)


Processing playlist: 37i9dQZF1DX03b46zi3S82
Processing playlist: 37i9dQZF1DWSiZVO2J6WeI
Processing playlist: 37i9dQZF1DWYcDQ1hSjOpY
Processing playlist: 37i9dQZF1DX2fOHhJX6Bvb
Processing playlist: 37i9dQZF1DWWSads6V2oIk
Processing playlist: 37i9dQZF1DX9NmDLwNQnXE
Processing playlist: 37i9dQZF1DWXzR2GKEiHgT
Processing playlist: 37i9dQZF1DX709Kv0V067V
Processing playlist: 37i9dQZF1DX4WiXBXnxjSY
Processing playlist: 37i9dQZF1DX9if5QDLdzCa
Processing playlist: 37i9dQZF1DX5FuBDzVtEFX
Processing playlist: 37i9dQZF1DXdbkmlag2h7b
Processing playlist: 37i9dQZF1DWYoDXiQsd3D2
Processing playlist: 37i9dQZF1DX7FUaCNVd2fs
Processing playlist: 37i9dQZF1DWUm4vT7WQxcD
Processing playlist: 37i9dQZF1DWWFimZsaMuv4
Processing playlist: 37i9dQZF1DX9mxKt6WorzQ
Processing playlist: 37i9dQZF1DWVs65pqQR22I
Processing playlist: 37i9dQZF1DXc1Etiiw8agT
Processing playlist: 37i9dQZF1DX2WtjAdkiMga
Processing playlist: 37i9dQZF1DX8skPjZYk8mL
Processing playlist: 37i9dQZF1DX2UkbeRPWQqZ
Processing playlist: 37i9dQZF1DX

In [16]:
# Enfants
category_id = "0JQ5DAqbMKFFoimhOqWzLB"

filtered_df = df[df['category_id'] == category_id]
playlist_ids = filtered_df['playlist_id'].tolist()

df_enfants = get_all_playlist_track_info(playlist_ids)
df_enfants.to_excel("df_enfants.xlsx", index = False)


Processing playlist: 37i9dQZF1DWYa24lU2SeaC
Processing playlist: 37i9dQZF1DWUij1y62UPcn
Processing playlist: 37i9dQZF1DX8oWCUlN9CxD
Processing playlist: 37i9dQZF1DX4nQ5NlnrDCH
Processing playlist: 37i9dQZF1DWY1vQTCT1CCD
Processing playlist: 37i9dQZF1DX0BoZKLoqg4z
Processing playlist: 37i9dQZF1DWYPUCdJXtD7j


In [17]:
# En voiture
category_id = "0JQ5DAqbMKFIRybaNTYXXy" 

filtered_df = df[df['category_id'] == category_id]
playlist_ids = filtered_df['playlist_id'].tolist()

df_voiture = get_all_playlist_track_info(playlist_ids)
df_voiture.to_excel("df_voiture.xlsx", index = False)


Processing playlist: 37i9dQZF1DX1S89tRXh2OS
Processing playlist: 37i9dQZF1DWWo4kzWesMJm
Processing playlist: 37i9dQZF1DXcwnhpAC5MCO
Processing playlist: 37i9dQZF1DX5rT0k5ulJ5y
Processing playlist: 37i9dQZF1DX4iPyiZG8RHQ
Processing playlist: 37i9dQZF1DWVuV87wUBNwc
Processing playlist: 37i9dQZF1DXcSPhLAnCjoM
Processing playlist: 37i9dQZF1DXbS5WTN5nKF7
Processing playlist: 37i9dQZF1DWYAaAF4glatK
Processing playlist: 37i9dQZF1DWVAWlq3l00p0
Processing playlist: 37i9dQZF1DXd0Y4aXXQXWv
Processing playlist: 37i9dQZF1DWWMOmoXKqHTD
Processing playlist: 37i9dQZF1DX9wC1KY45plY
Processing playlist: 37i9dQZF1DWU13kKnk03AP
Processing playlist: 37i9dQZF1DXdOEFt9ZX0dh
Processing playlist: 37i9dQZF1DXdfhOsjRMISB
Processing playlist: 37i9dQZF1DX2ogDiL6nZJr
Processing playlist: 37i9dQZF1DWYs83FtTMQFw
Processing playlist: 37i9dQZF1DWTkxQvqMy4WW


In [ ]:
category_id = "0JQ5DAqbMKFDkd668ypn6O" # Metal

filtered_df = df[df['category_id'] == category_id]
playlist_ids = filtered_df['playlist_id'].tolist()

df_metal = get_all_playlist_track_info(playlist_ids)

df_metal.to_excel("df_metal.xlsx", index = False)


In [ ]:
category_id = "0JQ5DAqbMKFAJ5xb0fwo9m" # Jazz

filtered_df = df[df['category_id'] == category_id]
playlist_ids = filtered_df['playlist_id'].tolist()

df_jazz = get_all_playlist_track_info(playlist_ids)

df_jazz.to_excel("df_jazz.xlsx", index = False)


In [ ]:
category_id = "0JQ5DAqbMKFQIL0AXnG5AK" # Tendances

filtered_df = df[df['category_id'] == category_id]
playlist_ids = filtered_df['playlist_id'].tolist()

df_tendances = get_all_playlist_track_info(playlist_ids)

df_tendances.to_excel("df_tendances.xlsx", index = False)


In [ ]:
category_id = "0JQ5DAqbMKFPw634sFwguI" # Equal

filtered_df = df[df['category_id'] == category_id]
playlist_ids = filtered_df['playlist_id'].tolist()

df_equal = get_all_playlist_track_info(playlist_ids)

df_equal.to_excel("df_equal.xlsx", index = False)


In [ ]:
category_id = "0JQ5DAqbMKFPrEiAOxgac3" # Classique

filtered_df = df[df['category_id'] == category_id]
playlist_ids = filtered_df['playlist_id'].tolist()

df_classique = get_all_playlist_track_info(playlist_ids)

df_classique.to_excel("df_classique.xlsx", index = False)


In [ ]:
category_id = "0JQ5DAqbMKFy78wprEpAjl" # Folk

filtered_df = df[df['category_id'] == category_id]
playlist_ids = filtered_df['playlist_id'].tolist()

df_folk = get_all_playlist_track_info(playlist_ids)

df_folk.to_excel("df_folk.xlsx", index = False)


In [ ]:
category_id = "0JQ5DAqbMKFCbimwdOYlsl" # Focus

filtered_df = df[df['category_id'] == category_id]
playlist_ids = filtered_df['playlist_id'].tolist()

df_focus = get_all_playlist_track_info(playlist_ids)

df_focus.to_excel("df_focus.xlsx", index = False)


In [ ]:
category_id = "0JQ5DAqbMKFIpEuaCnimBj" # Soul

filtered_df = df[df['category_id'] == category_id]
playlist_ids = filtered_df['playlist_id'].tolist()

df_soul = get_all_playlist_track_info(playlist_ids)

df_soul.to_excel("df_soul.xlsx", index = False)


In [ ]:
category_id = "0JQ5DAqbMKFEOEBCABAxo9" # Netflix

filtered_df = df[df['category_id'] == category_id]
playlist_ids = filtered_df['playlist_id'].tolist()

df_netflix = get_all_playlist_track_info(playlist_ids)

df_netflix.to_excel("df_netflix.xlsx", index = False)


In [ ]:
category_id = "0JQ5DAqbMKFFoimhOqWzLB" # Enfants

filtered_df = df[df['category_id'] == category_id]
playlist_ids = filtered_df['playlist_id'].tolist()

df_enfants = get_all_playlist_track_info(playlist_ids)

df_enfants.to_excel("df_enfants.xlsx", index = False)


In [ ]:
category_id = "0JQ5DAqbMKFJw7QLnM27p6" # Etudiants

filtered_df = df[df['category_id'] == category_id]
playlist_ids = filtered_df['playlist_id'].tolist()

df_etudiants = get_all_playlist_track_info(playlist_ids)

df_etudiants.to_excel("df_etudiants.xlsx", index = False)


In [ ]:
category_id = "0JQ5DAqbMKFLb2EqgLtpjC" # Bien-être

filtered_df = df[df['category_id'] == category_id]
playlist_ids = filtered_df['playlist_id'].tolist()

df_bienetre = get_all_playlist_track_info(playlist_ids)

df_bienetre.to_excel("df_bienetre.xlsx", index = False)


In [ ]:
category_id = "0JQ5DAqbMKFAjfauKLOZiv" # Punk

filtered_df = df[df['category_id'] == category_id]
playlist_ids = filtered_df['playlist_id'].tolist()

df_punk = get_all_playlist_track_info(playlist_ids)

df_punk.to_excel("df_punk.xlsx", index = False)


In [ ]:
category_id = "0JQ5DAqbMKFLjmiZRss79w" # Ambient

filtered_df = df[df['category_id'] == category_id]
playlist_ids = filtered_df['playlist_id'].tolist()

df_ambient = get_all_playlist_track_info(playlist_ids)

df_ambient.to_excel("df_ambient.xlsx", index = False)


In [ ]:
category_id = "0JQ5DAqbMKFKLfwjuJMoNC" # Country

filtered_df = df[df['category_id'] == category_id]
playlist_ids = filtered_df['playlist_id'].tolist()

df_country = get_all_playlist_track_info(playlist_ids)

df_country.to_excel("df_country.xlsx", index = False)


In [ ]:
category_id = "0JQ5DAqbMKFQiK2EHwyjcU" # Blues

filtered_df = df[df['category_id'] == category_id]
playlist_ids = filtered_df['playlist_id'].tolist()

df_blues = get_all_playlist_track_info(playlist_ids)

df_blues.to_excel("df_blues.xlsx", index = False)


In [ ]:
category_id = "0JQ5DAqbMKFQ1UFISXj59F" # Arabe

filtered_df = df[df['category_id'] == category_id]
playlist_ids = filtered_df['playlist_id'].tolist()

df_arabe = get_all_playlist_track_info(playlist_ids)

df_arabe.to_excel("df_arabe.xlsx", index = False)


In [ ]:
category_id = "0JQ5DAqbMKFAQy4HL4XU2D" # Voyage

filtered_df = df[df['category_id'] == category_id]
playlist_ids = filtered_df['playlist_id'].tolist()

df_voyage = get_all_playlist_track_info(playlist_ids)

df_voyage.to_excel("df_voyage.xlsx", index = False)


In [ ]:
category_id = "0JQ5DAqbMKFF9bY76LXmfI" # Frequency

filtered_df = df[df['category_id'] == category_id]
playlist_ids = filtered_df['playlist_id'].tolist()

df_frequency = get_all_playlist_track_info(playlist_ids)

df_frequency.to_excel("df_frequency.xlsx", index = False)
